In [1]:
import helper as h

In [2]:
# st.set_page_config(layout="wide")


TZ = "Europe/Berlin"  # 'America/Los_Angeles'
TARGET_DELTA_FASTING = (3, 4)
TARGET_DELTA_FIRST_MEAL = (1, 1)
TARGET_DELTA_LAST_MEAL = (2, 3)
TARGET_DELTA_SLEEP = (6.5, 7)
DT_DEEP_FAST_IN_HOURS = 12  # this is the time after which the deep fasting state starts
MIN_GAP_BETWEEN_SESSIONS_IN_MINUTES = DT_DEEP_FAST_IN_HOURS * 60
ROLLING_WINDOW_DAYS = 14
GAMMA = 0.9

In [ ]:
def load_data():

    # import pickle
    # with open("dev_data.pkl", "rb") as f:
    #     st.warning("Dev Data is used...")
    #     ret = pickle.load(f)

    # ret["df_score"] = h.calculate_score(
    #     ret["df_deep_fast_viz"],
    #     ret["df_first_meal_viz"],
    #     ret["df_last_meal_viz"],
    #     ret["df_sleep_duration_viz"],
    #     target_delta_fasting = TARGET_DELTA_FASTING,
    #     target_delta_first_meal = TARGET_DELTA_FIRST_MEAL,
    #     target_delta_last_meal = TARGET_DELTA_LAST_MEAL,
    #     target_delta_sleep = TARGET_DELTA_SLEEP,
    #     rolling_window_days=ROLLING_WINDOW_DAYS,
    #     gamma=GAMMA
    # )

    # return ret

    # load sleep data
    df_sleep = h.load_sleep_data_from_supabase()
    df_sleep_sessions = h.identify_sessions(
        df_sleep,
        min_gap_between_sessions_in_minutes=30,
        min_duration_of_session_in_minutes=60,
        add_sleep_duration_in_hours=True,
    )

    # load food data
    df_eat = h.load_eat_data_from_supabase()
    df_eat_sessions = h.identify_sessions(
        df_eat,
        min_gap_between_sessions_in_minutes=MIN_GAP_BETWEEN_SESSIONS_IN_MINUTES,
        min_duration_of_session_in_minutes=0,
    )

    # evaluate deep fasting sessions
    df_deep_fast_sessions = h.evaluate_deep_fast_sessions(
        df_eat_sessions, DT_DEEP_FAST_IN_HOURS
    )

    # evaluate delta to first and last meal
    df_first_and_last_meal = h.evaluate_delta_to_first_and_last_meal(
        df_sleep_sessions, df_eat
    )

    # evaluate data for visualization
    df_sleep_sessions_viz = h.process_for_visualization(df_sleep_sessions, TZ)
    df_eat_viz = h.process_for_visualization(df_eat, TZ)
    df_eat_sessions_viz = h.process_for_visualization(df_eat_sessions, TZ)
    df_deep_fast_sessions_viz = h.process_for_visualization(df_deep_fast_sessions, TZ)

    df_deep_fast_viz = h.process_deep_fast_sessions_for_viz(
        df_deep_fast_sessions, timezone=TZ
    )
    df_first_meal_viz, df_last_meal_viz = h.process_first_and_last_meal_data_for_viz(
        df_first_and_last_meal, timezone=TZ
    )
    df_sleep_duration_viz = h.process_sleep_sessions_for_viz(df_sleep_sessions, TZ)

    # calculate score
    df_score, debug_info = h.calculate_score(
        df_deep_fast_viz,
        df_first_meal_viz,
        df_last_meal_viz,
        df_sleep_duration_viz,
        target_delta_fasting=TARGET_DELTA_FASTING,
        target_delta_first_meal=TARGET_DELTA_FIRST_MEAL,
        target_delta_last_meal=TARGET_DELTA_LAST_MEAL,
        target_delta_sleep=TARGET_DELTA_SLEEP,
        rolling_window_days=ROLLING_WINDOW_DAYS,
        gamma=GAMMA,
    )

    return {
        "df_sleep": df_sleep,
        "df_sleep_sessions": df_sleep_sessions,
        "df_eat": df_eat,
        "df_eat_sessions": df_eat_sessions,
        "df_deep_fast_sessions": df_deep_fast_sessions,
        "df_first_and_last_meal": df_first_and_last_meal,
        "df_sleep_sessions_viz": df_sleep_sessions_viz,
        "df_eat_viz": df_eat_viz,
        "df_eat_sessions_viz": df_eat_sessions_viz,
        "df_deep_fast_sessions_viz": df_deep_fast_sessions_viz,
        "df_deep_fast_viz": df_deep_fast_viz,
        "df_first_meal_viz": df_first_meal_viz,
        "df_last_meal_viz": df_last_meal_viz,
        "df_sleep_duration_viz": df_sleep_duration_viz,
        "df_score": df_score,
        "debug_info": debug_info,
    }
    
load_data()

In [4]:
df_eat = h.load_eat_data_from_supabase()

In [ ]:
df_eat